In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows
rl.head()


Metadata file is not present in the given path; proceeding to read the csv file.


,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,ochlockonee moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,nelson's spiny pocket mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,comoro friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"atlantic halibut, halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic å_åÑåÐ northeast; Atlantic å_åÑåÐ northwest</div>; Canada (Newfoundland I); Denmar...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,oktibbeha riverlet crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [2]:
# CME
em.get_key(rl) # get key

# Note: The id column of the  redlist csv doesn't actually look like the object created here (rl).  In the csv,
# id numbers jump around since they represent original row number of the original scraped data csv.  Apparently
# this library can detect what we want and enumerate things in order if you give it a key feature like this.


'id'

In [3]:
#Read in ARKIVE
# Read in csv as dataframe
ar = em.read_csv_metadata("finalArkive.csv", encoding="ISO-8859-1", key="id")

# glance at first few rows
ar.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount,name
0,0,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula,Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0,alcatraz de abbott; piquero de abbott; abbott's booby
1,1,AbbotÌ_å¢åÎÌ_åÎåÈs duiker,Cephalophus spadix,AbbottÌ_å¢åÎÌ_åÎåÈs duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 Ì_å¢åÎÌ_åÎÌ± 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1,abbotÌ_å¢åÎÌ_åÎåÈs duiker; abbottÌ_å¢åÎÌ_åÎåÈs duiker
2,2,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink
3,3,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0,adriatic trout; softmouth trout; adriatic salmon
4,4,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus,Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2,black-footed penguin; jackass penguin; african penguin


In [4]:
# Remove parentheticals from arkive common names
# author: CME
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['common_name'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'common_name'] = noparens
    else:
        error = error + 1

print(error)


818


In [5]:
# Concatenate nickname and common_name fields in arkive
# author: @andrewedstrom

# remove parenthesis contents
nameColumn = []
for index, row in ar.iterrows():
    nickname = str(row['scientific_name']) #look here if errors start happening
    common = str(row['common_name'])
    if re.sub('[^A-Za-z0-9]+', '', nickname) in re.sub('[^A-Za-z0-9]+', '', common): 
        nameColumn.append(nickname.lower())
    else:
        nameColumn.append((nickname + "; " + common).lower())
ar['name'] = nameColumn
ar.head()

,id,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount,name
0,0,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula,Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0,alcatraz de abbott; piquero de abbott; abbott's booby
1,1,AbbotÌ_å¢åÎÌ_åÎåÈs duiker,Cephalophus spadix,AbbottÌ_å¢åÎÌ_åÎåÈs duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 Ì_å¢åÎÌ_åÎÌ± 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1,abbotì_å¢åîì_åîåès duiker; abbottì_å¢åîì_åîåès duiker
2,2,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink
3,3,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo,80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0,adriatic trout; softmouth trout; adriatic salmon
4,4,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus,Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2,black-footed penguin; jackass penguin; african penguin


In [6]:
# Lowercase redlist name field
# author: @andrewedstrom
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in rl.iterrows():
    n = str(row['name'])
    rl.loc[index, 'name'] = n.lower()

rl.head()

,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,ochlockonee moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,nelson's spiny pocket mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,comoro friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"atlantic halibut, halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic å_åÑåÐ northeast; Atlantic å_åÑåÐ northwest</div>; Canada (Newfoundland I); Denmar...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,oktibbeha riverlet crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [7]:
# ar.to_csv('finalOutput.csv')

In [8]:
# Remove parentheticals from arkive genus
# author: @andrewedstrom
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['genus'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'genus'] = noparens
    else:
        error = error + 1

print(error)

818


In [9]:
#rl.to_csv('finalRedlist.csv')
#ar.to_csv('finalArkive.csv')

In [10]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = em.get_features_for_blocking(rl, ar)


In [11]:
len(feature_table)
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x115a1b488>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x115a041e0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x115a040d0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x115a04048>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,family_family_jac_qgm_3_qgm_3,family,family,qgm_3,qgm_3,jaccard,<function family_family_jac_qgm_3_qgm_3 at 0x1159e5ea0>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,family_family_cos_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,cosine,<function family_family_cos_dlm_dc0_dlm_dc0 at 0x1159e5ae8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,family_family_jac_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,jaccard,<function family_family_jac_dlm_dc0_dlm_dc0 at 0x1159e5c80>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,family_family_mel,family,family,None,None,monge_elkan,<function family_family_mel at 0x1159e5a60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,family_family_lev_dist,family,family,None,None,lev_dist,<function family_family_lev_dist at 0x1159e5d08>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,family_family_lev_sim,family,family,None,None,lev_sim,<function family_family_lev_sim at 0x10997b378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [12]:
#list name related features
feature_table[feature_table.left_attribute=='name']

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated


In [13]:
# CME -- I'm realizing now that we may want to start with the attribute equivalence
# blocker for names and see where that leaves us.  On my TO-DO.
equivB = em.AttrEquivalenceBlocker()
C0 = equivB.block_tables(rl, ar, 'name', 'name',
                    l_output_attrs=['name', 'genus'], 
                    r_output_attrs=['name', 'genus'])
C0

# lolwat this reveals zero matches?


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus


In [14]:
#Do overlap blocking on name to get candidate set
#Perform some kind of blocking to find candidate pairs
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'name', 'name', 
                     l_output_attrs=['name', 'genus'], 
                     r_output_attrs=['name', 'genus'])
C0

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
0,0,7316,0,coahuilix de hubbs snail,Coahuilix,alcatraz de abbott; piquero de abbott; abbott's booby,Papasula
1,1,4580,0,"carbonera pupfish, perrito de carbonera",Cyprinodon,alcatraz de abbott; piquero de abbott; abbott's booby,Papasula
2,2,7546,0,lago de las minas frog,Lithobates,alcatraz de abbott; piquero de abbott; abbott's booby,Papasula
3,3,4409,0,moravia de chirripo salamander,Bolitoglossa,alcatraz de abbott; piquero de abbott; abbott's booby,Papasula
4,4,7294,2,greater virgin islands skink,Spondylurus,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
5,5,1171,2,"gibbons' emo skink, viti barred treeskink",Emoia,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
6,6,4466,2,lower cerro pigmy salamander,Thorius,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
7,7,4816,2,blue mountain water skink,Eulamprus,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
8,8,7616,2,"cerro san felipe pigmy salamander, upper cerro pigmy salamander",Thorius,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
9,9,4572,2,two-fingered skink,Chalcides,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua


In [15]:
#Only keep candidate pairs which share genus
ab = em.AttrEquivalenceBlocker()
C1 = ob.block_candset(C0, 'genus', 'genus')
C1

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


,_id,ltable_id,rtable_id,ltable_name,ltable_genus,rtable_name,rtable_genus
41,41,1470,2,"adelaide pigmy blue-tongue skink, pygmy bluetongue",Tiliqua,adelaide pigmy blue-tongue skink; pygmy bluetongue; adelaide pygmy bluetongue skink,Tiliqua
50,50,1192,3,adriatic salmon,Salmo,adriatic trout; softmouth trout; adriatic salmon,Salmo
58,58,2017,5,"african teak, afromosia, afrormosia",Pericopsis,afrormosia; assamela; kokrodua; african teak,Pericopsis
173,173,7543,8,karpathos frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
318,318,4363,8,albanian water frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
437,437,7543,9,karpathos frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
582,582,4363,9,albanian water frog,Pelophylax,albanian frog; albanian pool frog; balkan frog; virpazar frog; albanian water frog,Pelophylax
890,890,339,12,blind swamp eel,Ophisternon,blind swamp eel; anguila ciega,Ophisternon
899,899,661,13,decary's leaf chameleon,Brookesia,armoured chameleon; stump tailed chameleon; antsingy leaf chameleon,Brookesia
910,910,4575,16,aran rock lizard,Iberolacerta,lagartija aranesa; aran rock lizard,Iberolacerta


In [16]:
#For shits and giggles: 